In [1]:
import os
import numpy as np
import pandas as pd
import pickle as pk
from Symbol import Symbol
from multiprocessing import Pool
import time

In [2]:
def load_debbuger(path,y):
    print(path+y)
    return pk.load(open(path+y,'rb'))

In [7]:
class Merger:
    def string_to_epoch(timestamp,arr, n):
        [date,t]=timestamp.split()
        date=[int(x) for x in date.split('-')]
        t=[int(x) for x in t.split(':')]

        if((date[1]<=3 and date[2]<=29 and t[0]<=2 and t[1]<=0)):
            tmz='CET'
        elif(date[1]>=10 and date[2]>=29 and t[0]>=3 and t[1]>=0):
            tmz='CET'
        elif(date[1]>=10 and date[2]>=29):
            try:
                prev=arr[n-12]
                [pdate,ptime]=prev.split()
                pdate=[int(x) for x in pdate.split('-')]
                ptime=[int(x) for x in ptime.split(':')]
                if(t[0]==ptime[0]):
                    tmz='CET'
                else:
                    tmz='CEST'
            except:
                futr=arr[n+12]
                [fdate,ftime]=prev.split()
                fdate=[int(x) for x in fdate.split('-')]
                ftime=[int(x) for x in ftime.split(':')]
                if(t[0]==ftime[0]):
                    tmz='CEST'
                else:
                    tmz='CET'
        else:
            tmz='CEST'


        return int(time.mktime(time.strptime(timestamp+' '+tmz,'%Y-%m-%d %H:%M %Z')))
    def load(path):
        files=[y for y in os.listdir(path)  if '.pkl' in y]
        objects=[pk.load(open(path+y,'rb'))[0] for y in files]
        data=[[[pd.DataFrame(x.data).transpose(),x.tick,pd.DataFrame(x.data).transpose().index]for x in y] for y in objects]
        datadic={}
        for file in data:
            for symbol in file:
                index=[Merger.string_to_epoch(x,symbol[0].index.tolist(),n) for x,n in zip(symbol[0].index.tolist(), [n for n in range(len(symbol[0].index.tolist()))])]
                symbol[0].index=index
                if(symbol[1] not in datadic.keys()):
                    datadic[symbol[1]]=[symbol[0]]
                else:
                    datadic[symbol[1]]+=[symbol[0]]
        return datadic
    def merge_ticker(arg):
        [arr,ticker]=arg
        columns=[ticker+' '+x.replace('1.','').replace('2.','').replace('3.','').replace('4.','').replace('5.','').replace(' M','M') for x in arr[0].columns]
        df=pd.DataFrame(columns=columns)
        for file in arr:
                for index in file.index:
                    if(index not in df.index):
                        df=df.append(pd.DataFrame(data=[file.loc[index].tolist()],index=[index],columns=columns))
        print(True in df.duplicated())
        return df.sort_index()
    def gen_df(path):
        startt=time.time()
        print('loading')
        data = Merger.load(path)
        tickers=list(data.keys())
        ltime=time.time()
        print(ltime-startt)
        print('merging tickers')
        with Pool(4) as p:
            dfs=p.map(Merger.merge_ticker,zip([data[ticker] for ticker in tickers],tickers))

        print(time.time()-ltime)
        return dfs

In [8]:
dataframes=Merger.gen_df('./data/')

loading
26.432520151138306
merging tickers
False
False
False
False
False
False
False
False
False
98.04589796066284


In [10]:
dataframes[0]

,TSLA open,TSLA high,TSLA low,TSLA close,TSLA volume,TSLA MACD_Hist,TSLA MACD,TSLA MACD_Signal
1582539600,831.4800,834.4992,829.2400,834.4700,91089,-8.6496,-1.4541,-7.1954
1582539900,834.5000,835.5800,831.6000,831.6900,88094,-8.6334,-0.9075,-7.7259
1582540200,832.0400,837.3900,831.6100,834.0000,95825,-8.5724,-0.3989,-8.1734
1582540500,834.0000,836.0000,832.5100,833.1300,82419,-8.1803,0.2399,-8.4202
1582540800,833.2503,841.5000,833.1200,839.5079,173332,-7.6089,0.8527,-8.4617
...,...,...,...,...,...,...,...,...
1586266800,545.2150,548.4400,544.3027,547.6637,169720,-0.0211,-3.5738,-3.5527
1586267100,548.0000,549.8600,546.4201,548.1500,137629,0.1058,-3.3868,-3.4926
1586267400,548.4800,551.9400,547.8810,551.9400,205900,0.3791,-2.9941,-3.3732
1586267700,550.0000,552.0000,544.0350,547.3300,265835,0.8577,-2.3493,-3.2070
